In [15]:
import mysql.connector
import os

mysql_pw = os.environ.get('MYSQL')
rows = []

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=mysql_pw,
    database="job_matcher",
)

try:
    with conn.cursor(dictionary=True) as cursor:
        sql = """
        SELECT *
        FROM job_postings
        WHERE content != ""
        LIMIT 10;
        """
        cursor.execute(sql)
        rows = cursor.fetchall()
finally:
    cursor.close()
    conn.close()

#rows[0]['content']

'삼성카드고객서비스㈜\nVVIP 고객 여행상담(심야)\n삼성카드고객서비스는 카드발급, 대출,\r\n고객상담서비스 업무 등을 담당하고\r\n있는 고객관계관리 전문회사 입니다.\r\n\r\n오늘날 카드 결제는 민간소비의 50%\r\n이상을 차지할 정도로 비약적인 발전을\r\n해왔습니다.\r\n\r\n고객 여러 분의 건전하고 합리적인\r\n소비를 지원하기 위해 노력해 온 카드\r\n서비스 영역이 있기에 가능한 일이었습니다.\r\n\r\n앞으로도 삼성카드고객서비스는\r\n고객 존중의 마음가짐으로 종합적인\r\n서비스 제공을 통해 업계의 변화와\r\n혁신을 선도해 나가겠습니다.\n포지션 및 자격요건\n포지션\n( 1명 )\nㅁ 모집직무 : 인바운드 여행상담 심야(ATR)\n※ 카드상담 일부포함\n※ 계약직(1년 단위 최대 2년) 후 정규전환 기회 부여\nㅁ 근무시간\n- [평일] 22:00~08:30(9.5h), [휴일] 20:00~08:30(11h)\n※ 스케쥴 근무로 한달 中 10일 내외 출근(평일, 휴일 포함)\nㅁ 근무지역\n- 서울 구로구 새말로97 (신도림테크노마트)\nㅁ 처우수준\n- 월 296만원 ~ 331만원(월 296만원 + 월 추가 인센티브 0~35만원)\n※ 총연봉 : 3,552만원 ~ 4,394만원(기본연봉 3,552만원 + 월 인센티브 최대 180만원 + 상/하반기 성과급 최대 662만원)\n※ 심야근무 시, 심야수당 월 15만원 별도지급\nㅁ 복리후생(근속년수 및 연령에 따라 차등지급)\n- 경조금, 경조화환, 경조휴가 지원\n(부모/본인 생일 케이크 및 본인생일 복지포인트 지급)\n- 콘도지원, 캐리비안베이 쿠폰 지원, 결혼도움방(예식장)\n- 정규직 대상 주택전세 및 주택구입 대출지원\n- 본인/배우자/자녀 실손보험 가입 및 의료비 지원, 임직원 단체보험 가입\n- 정규직 복지포인트 년 100만원 지급\nㅁ 자격조건\n- 경력 : 최소 1~3년 이상의 국제선 항공권 및 국내/외 호텔 예약 경력\n- 활용 프로그램 : 토파스 / 아바쿠스\n

In [9]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema.runnable import RunnableLambda
api_key = os.environ.get("OPENAI_API_KEY")

model = ChatOpenAI(
    model = 'gpt-4.1',
    temperature = 0.1
)

response_schemas = [
    ResponseSchema(name="근무조건", description="간결한 근무조건 요약"),
    ResponseSchema(name="복리후생", description="간결한 복리후생 요약"),
    ResponseSchema(name="자격", description="간결한 자격요건 요약"),
    ResponseSchema(name="전형", description="간결한 전형절차 요약"),
    ResponseSchema(name="기타", description="기타 지원방법과 문의처 요약")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

for job_content in rows:
    prompt = ChatPromptTemplate.from_template(
        """
        너는 채용공고 요약 전문가야.
        다음 채용공고 본문을 읽고, 주어진 형식에 따라 JSON으로 간결하게 요약해줘.
        {format_instructions}

        채용공고 본문:
        \"\"\"
        {job_content}
        \"\"\"
        """
    )

    chain = (
        prompt
        | model
        | RunnableLambda(lambda x: output_parser.parse(x.content))
    )

    result = chain.invoke({
        "format_instructions" : format_instructions,
        "job_content" : job_content
    })

    print(result)
    print('*' * 20)

{'근무조건': '계약직(1년 단위, 최대 2년 후 정규전환 기회), 심야 인바운드 여행상담(카드상담 일부 포함), 스케줄 근무(월 10일 내외 출근), 평일 22:00~08:30, 휴일 20:00~08:30, 서울 구로구 신도림테크노마트 근무, 월 296~331만원(인센티브 포함), 심야수당 별도 지급.', '복리후생': '경조금/경조화환/경조휴가, 생일 케이크 및 복지포인트, 콘도·캐리비안베이 쿠폰, 결혼도움방, 정규직 대상 주택대출, 본인/가족 실손보험 및 의료비 지원, 임직원 단체보험, 정규직 복지포인트 연 100만원.', '자격': '국제선 항공권 및 국내/외 호텔 예약 경력 1~3년 이상, 토파스/아바쿠스 사용 가능자, VIP 고객상담·영어회화·카드사 여행서비스 상담 경력자 우대.', '전형': '서류전형 → 면접전형 → 채용검진 → 최종합격(면접일정 개별 통보).', '기타': '이메일(hyeonjeong74470.lee@samsung.com)로 이력서 제출, 상시채용, 응시분야 이력서에 명시, 허위사실 시 채용 취소, 채용문의 02-2000-8301, 채용서류 반환청구 가능(팩스 02-2000-8729).'}
********************
{'근무조건': '근무조건 미기재', '복리후생': '복리후생 미기재', '자격': '자격요건 미기재', '전형': '서류전형 > 1차면접 > 최종합격, 면접일정 추후 통보', '기타': '허위사실 발견 시 채용 취소, (주)김남주바이오 홈페이지 참고'}
********************
{'근무조건': '서울 성동구, 2025년 8월 1일~12월 31일(5개월), K-Star 고도화 구축 프로젝트 참여', '복리후생': '별도 언급 없음', '자격': '학력무관, 경력 6년 이상(중급), 웹스퀘어·JAVA 개발 필수, SQL(DB2) 중급, Proframe 경험자 우대', '전형': '서류전형 > 1차면접 > 2차면접 > 임원면접 > 최종합격(면접일정 추후 통보)', '기타': '허위사실 기재 시 